In [1]:
import numpy as np
import pandas as pd
import os
import openpyxl as xl
from openpyxl import load_workbook
from openpyxl.utils import FORMULAE

#change working directory
os.chdir('C:\\Users\\sdeshpande\\Desktop\\Fibre_sheath')

In [2]:
#save path and read excel
path = 'C:\\Users\\sdeshpande\\Desktop\\Fibre_sheath\\'
fibre_sheath = pd.read_excel(path+'FibreSheath.xlsx')

In [3]:
#delete columns 'ent_id' & 'id', if present
if 'ent_id' in fibre_sheath.columns:
    fibre_sheath.drop(['ent_id'], axis=1, inplace=True)
    
if 'id' in fibre_sheath.columns:
    fibre_sheath.drop(['id'], axis=1, inplace=True)
    
#rename column headers as required    
fibre_sheath.columns = ['label','type','calc_length','meas_length','from_','placement','backhaul']

In [4]:
#load excel using openpyxl library
workbook = xl.load_workbook('FibreSheath.xlsx')
#get the active worksheet
sheet_obj = workbook.active
#delete the active worksheet from the excel
workbook.remove_sheet(sheet_obj)
#write the pandas df that has updated column headers and columns to the excel file again
fibre_sheath.to_excel('FibreSheath.xlsx','FibreSheath', index = False)

c:\users\sdeshpande\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated function remove_sheet (Use wb.remove(worksheet) or del wb[sheetname]).
  


In [5]:
#set path variables
path1 = path + 'FibreSlack.xlsx'
path2 = path + 'FibreSheath.xlsx'

#load FibreSlack excel active worksheet
wb1 = xl.load_workbook(filename=path1)
ws1 = wb1.worksheets[0]

#load FibreSheath excel and create new worksheet for copying FibreSlack excel data
wb2 = xl.load_workbook(filename=path2)
ws2 = wb2.create_sheet('FibreSlack')

#copying data from FibreSlack to FibreSheath excel
for row in ws1:
    for cell in row:
        ws2[cell.coordinate].value = cell.value

#save FibreSheath excel
wb2.save(path2)

In [6]:
#open worksheet 
fibre_sheath = wb2["FibreSheath"]
max_rows = fibre_sheath.max_row

#apply formula for slack column
for row_num in range(2, max_rows+1):
    fibre_sheath['H%d'%row_num].value = '=COUNTIF(FibreSlack!B:B, FibreSheath!A%d)'%(row_num)

#save FibreSheath excel 
wb2.save(filename="FibreSheath.xlsx")

In [7]:
#split the worksheet into multiple ribbon worksheets
dfs = pd.read_excel(path + 'FibreSheath.xlsx')

def nd(df):
    my_list = dfs['label'].str[4].unique()
    od = dict()
    for i in my_list:
        od[i] = df[df['label'].str[4] == i]
    return od

from openpyxl import load_workbook
my_dict = nd(dfs)

def df_excel(k):
    book = load_workbook(path + 'FibreSheath.xlsx')
    writer = pd.ExcelWriter(path + 'FibreSheath.xlsx', engine = 'openpyxl')
    writer.book = book
    my_dict[k].sort_values('label').to_excel(writer, sheet_name = k, index=False)
    writer.save()
    writer.close()
    return None
    
for k,v in my_dict.items():
    df_excel(k)